In [14]:
import os
import numpy as np
from lime.lime_image import LimeImageExplainer
from PIL import Image
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Define paths (adjust these based on your actual paths)
base_path = r'C:\Users\dunli\Documents\STSY-project-main'
model_weights_path = os.path.join(base_path, 'Training_Code', 'runs', 'detect', 'train16', 'weights', 'best.pt')
image_path = os.path.join(base_path, 'Training Data', 'test', 'images')
save_path = os.path.join(base_path, 'Lime tested image')

# Load the YOLOv5 model
model = YOLO(model_weights_path)  # Replace with your actual model path

# Function to preprocess images
def preprocess_image(image_path):
    try:
        image = Image.open(image_path).convert("RGB")  # Ensure image is in RGB format
        resized_image = image.resize((640, 640))  # Resize to model input size
        return np.array(resized_image)
    except Exception as e:
        print(f"Error in preprocessing image {image_path}: {str(e)}")
        return None

# Function to predict with the YOLO model
def predict(input_image):
    results = model(input_image, imgsz=640)  # Adjust size as per your model's input size requirement
    # Assuming results is a list containing detection results, and you want the first result
    output = results.pandas().xyxy[0].to_numpy()  # Convert to numpy array
    return output

# Function to generate LIME explanations for images in a batch
def generate_lime_batch(images, save_folder):
    os.makedirs(save_folder, exist_ok=True)
    for image_file, image in images.items():
        try:
            image_name, ext = os.path.splitext(image_file)

            # Create the LIME explainer
            explainer = LimeImageExplainer()

            # Explain the model's predictions for the image
            explanation = explainer.explain_instance(
                image,  # Pass the image
                predict,
                top_labels=5,
                num_samples=1000,
            )

            # Get the image and mask for the explanation
            lime_image, mask = explanation.get_image_and_mask(
                explanation.top_labels[0],
                positive_only=False,
                num_features=10,
                hide_rest=False,
                min_weight=0.01  # Adjust as per your preference
            )

            # Normalize the image to the [0, 1] range
            lime_image = (lime_image - np.min(lime_image)) / (np.max(lime_image) - np.min(lime_image))

            # Save the LIME image
            lime_image_path = os.path.join(save_folder, f"{image_name}_lime.jpg")
            plt.imsave(lime_image_path, lime_image)

            print(f"Processed {image_file}")

        except Exception as e:
            print(f"Error processing {image_file}: {str(e)}")

# Function to generate LIME explanations for all images in a directory
def generate_lime_for_folder(image_folder, save_folder, batch_size=10):
    if not os.path.exists(image_folder):
        raise FileNotFoundError(f"The directory '{image_folder}' does not exist or is inaccessible.")

    os.makedirs(save_folder, exist_ok=True)

    images = {}
    for image_file in os.listdir(image_folder):
        try:
            image_file_path = os.path.join(image_folder, image_file)

            # Preprocess the image
            image = preprocess_image(image_file_path)
            if image is None:
                continue

            images[image_file] = image

            # Process batch of images
            if len(images) >= batch_size:
                generate_lime_batch(images, save_folder)
                images = {}

        except Exception as e:
            print(f"Error processing {image_file}: {str(e)}")

    # Process any remaining images in the last batch
    if images:
        generate_lime_batch(images, save_folder)

# Usage example: Generate LIME explanations for all images in the specified folder
generate_lime_for_folder(image_folder=image_path, save_folder=save_path, batch_size=10)



  0%|          | 0/1000 [00:00<?, ?it/s]


Error processing pos_0_frame_15_4_aug_5.png: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\core\src\copy.cpp:1026: error: (-215:Assertion failed) top >= 0 && bottom >= 0 && left >= 0 && right >= 0 && _src.dims() <= 2 in function 'cv::copyMakeBorder'



  0%|          | 0/1000 [00:00<?, ?it/s]


Error processing pos_0_frame_1_1_aug_8.png: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\core\src\copy.cpp:1026: error: (-215:Assertion failed) top >= 0 && bottom >= 0 && left >= 0 && right >= 0 && _src.dims() <= 2 in function 'cv::copyMakeBorder'



  0%|          | 0/1000 [00:00<?, ?it/s]


Error processing pos_0_frame_1_2_aug_1.png: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\core\src\copy.cpp:1026: error: (-215:Assertion failed) top >= 0 && bottom >= 0 && left >= 0 && right >= 0 && _src.dims() <= 2 in function 'cv::copyMakeBorder'



KeyboardInterrupt: 

In [ ]:
import os
import cv2
from lime import lime_image
from ultralytics import YOLO  # Replace with UltraLytics import method

# Load UltraLytics YOLOv5 model
def load_ultralytics_model(weights_path):
    model = Model(weights_path)
    return model

# Function to perform YOLOv5 prediction using UltraLytics
def ultralytics_yolo_predict(image_path, model):
    img = cv2.imread(image_path)  # Load image
    # Process image to fit UltraLytics YOLOv5 input requirements, e.g., resizing, normalization
    results = model(img)  # Perform prediction
    return results.xyxy[0]  # Assuming UltraLytics provides bounding box coordinates directly

# Function to use LIME for explaining UltraLytics YOLOv5 predictions
def explain_ultralytics_yolo_prediction(image_path, model):
    explainer = lime_image.LimeImageExplainer()

    # Function to get predictions from UltraLytics YOLOv5 model
    def ultralytics_yolo_predict_fn(images):
        return ultralytics_yolo_predict(images, model)

    # Explain prediction
    explanation = explainer.explain_instance(image_path, ultralytics_yolo_predict_fn, top_labels=5, hide_color=0, num_samples=1000)

    return explanation

# Folder containing test images
folder_path = r'C:\Users\dunli\Documents\STSY-project-main\Training Data\test\images'

# List all files in the folder
image_files = os.listdir(folder_path)

# Load UltraLytics YOLOv5 model
ultralytics_yolov5_model = load_ultralytics_model('path_to_ultralytics_weights.pt')

# Iterate through each image file
for image_file in image_files:
    if image_file.endswith('.jpg') or image_file.endswith('.png'):  # Adjust based on your image formats
        image_path = os.path.join(folder_path, image_file)
        print(f"Explaining predictions for {image_path}...")
        explanation = explain_ultralytics_yolo_prediction(image_path, ultralytics_yolov5_model)
        # Optionally, you can save or display the explanation here
        # explanation.show_in_notebook()
        # Save explanation as needed
        explanation.save_to_file(f'explanation_{image_file}.html')